In [13]:
import pandas as pd
import os
import glob
from datetime import datetime


folderPath = 'E:/Evaidya/PAC/Nerali'
pacName = 'SHRI BASAVESHWAR PKPS NERALI.xlsx'
pacIdKey = '91150306096003'
# Read all sheets into a dictionary of dataframes
sheetsDict = pd.read_excel(f'{folderPath}/{pacName}', sheet_name=None)

#pd.set_option('display.float_format', lambda x: '%.9f' % x)
column_name = 'AdmissionNo'
admissionDate = 'AdmissionDate'
sanctionDate = 'SanctionedDate'
depositDate = 'DepositDate'
transactionDate = 'TransactionDate'
basicValiidationFailPath = f'{folderPath}/BasicValidationFail'
uploadTemplatePath = f'{folderPath}/UploadTemplates'

if not os.path.exists(basicValiidationFailPath):
    os.makedirs(basicValiidationFailPath)

if not os.path.exists(uploadTemplatePath):
    os.makedirs(uploadTemplatePath)


def generateSbTemplate(df, templateName):
    df_sb = df
    df_sb['IntroducerNo'] = 0
    df_sb['ChequeOption'] = 'No'
    df_sb['JointAdmissionNo'] = 0
    df_sb['RegisterSlNo'] = 0
    df_sb['PacsIDPkey'] = pacIdKey
    
    if 'LedgerFolioNo' not in df_sb.columns:
        df_sb['LedgerFolioNo'] = ''

    df_sb['DepositDate'] = pd.to_datetime(df_sb['DepositDate']).dt.strftime('%d-%m-%Y')
    df_sb = df_sb[['AdmissionNo','ProductDescription','AccountNo','DepositDate','IntroducerNo','ChequeOption','OperationTypeDesc','IsOrganisation','RegisterSlNo','Balance','LedgerFolioNo','JointAdmissionNo','PacsIDPkey']]
    
    df_sb.to_excel(f'{uploadTemplatePath}/{templateName}_Upload_template.xlsx', index=False, sheet_name='Deposit')
    return print('Complete')

def generatePigmyTemplate(df, templateName):
    df_pigmy_upload_deposits = df
    df_pigmy_upload_deposits['AgentNo'] = 1
    df_pigmy_upload_deposits['PeriodinYears'] = 1    
    df_pigmy_upload_deposits['PacsIDPKey'] = pacIdKey
    df_pigmy_upload_deposits['BranchId'] = ''
    df_pigmy_upload_deposits['IsPayable'] = '0'
    df_pigmy_upload_deposits['TotalInterestAmount'] = '0'
    
    if 'LedgerFolioNo' not in df_pigmy_upload_deposits.columns:
        df_pigmy_upload_deposits['LedgerFolioNo'] = ''

    if 'RateOfInterest' not in df_pigmy_upload_deposits.columns and 'ROIPercentage' in df_pigmy_upload_deposits.columns:
        df_pigmy_upload_deposits['RateOfInterest'] = df_pigmy_upload_deposits['ROIPercentage']

    df_pigmy_upload_deposits.loc[df_pigmy_upload_deposits['RateOfInterest'] < 1, 'RateOfInterest'] *= 100
    
    df_pigmy_upload_deposits['DepositDate'] = pd.to_datetime(df_pigmy_upload_deposits['DepositDate']).dt.strftime('%d-%m-%Y')
    df_pigmy_upload_deposits['MaturityDate'] = pd.to_datetime(df_pigmy_upload_deposits['MaturityDate']).dt.strftime('%d-%m-%Y')
    df_pigmy_upload_deposits = df_pigmy_upload_deposits[['AdmissionNo','ProductDescription','AccountNo','DepositDate','OperationTypeDesc','InstallmentAmount','PeriodinMonths','PeriodinYears','MaturityDate','TotalAmount','RateOfInterest','AgentNo','FrequencyDescription','LedgerFolioNo','IsPayable','TotalInterestAmount','PacsIDPKey','BranchId']]
    
    df_pigmy_upload_deposits.to_excel(f'{uploadTemplatePath}/{templateName}_Upload_template.xlsx', index=False, sheet_name='PigmyDeposits')
    return print('Completed')


def generateFdNonTemplate(df, templateName):
    df_fd = df
    print(df_fd.columns)
    df_fd['InstallmentAmount'] = df_fd['InterestAmount']
    df_fd['InstallmentsPaid'] = ''
    df_fd['TotalInstallmentAmountPaid'] = 0
    df_fd['LastPaidInstallDate'] = ''
    df_fd['chkIsInterestPostingToCB'] = 0
    df_fd['JointAdmissionNo'] = 0
    df_fd['Status'] = 'Active'
    df_fd['PacsIDPkey'] = pacIdKey
    
    if 'LedgerFolioNo' not in df_fd.columns:
        df_fd['LedgerFolioNo'] = ''

    if 'TerminMonths' not in df_fd.columns and 'PeriodinMonths' in df_fd.columns:
        df_fd['TerminMonths'] = df_fd['PeriodinMonths']

    df_fd['DepositDate'] = pd.to_datetime(df_fd['DepositDate']).dt.strftime('%d-%m-%Y')
    df_fd['MaturityDate'] = pd.to_datetime(df_fd['MaturityDate']).dt.strftime('%d-%m-%Y')
    df_fd = df_fd[['ProductDescription','AdmissionNo','AccountNo','OperationTypeDesc','DepositDate','DepositAmount','TerminDays','TerminMonths','MaturityDate','MaturityAmount','RateOfInterest','Status','PacsIDPkey','LedgerFolioNo','DepositTypeDesc','InterestAmount','InstallmentAmount','InterestPaymentModeDesc','InstallmentsPaid','TotalInstallmentAmountPaid','LastPaidInstallDate','IsInterestPosted','chkIsInterestPostingToCB','LastInterestPostingDate','TotalInterestAmount','JointAdmissionNo']]
    
    df_fd.to_excel(f'{uploadTemplatePath}/{templateName}_Upload_template.xlsx', index=False, sheet_name='FDNonCumulative')    
    return print('Completed')

def generateFdComTemplate(df, templateName):
    df_fd_com = df        
    df_fd_com['chkIsInterestPostingToCB'] = 0
    df_fd_com['JointAdmissionNo'] = 0
    df_fd_com['Status'] = 'Active'
    df_fd_com['PacsIDPkey'] = pacIdKey
    
    if 'LedgerFolioNo' not in df_fd_com.columns:
        df_fd_com['LedgerFolioNo'] = ''

    if 'TerminMonths' not in df_fd_com.columns and 'PeriodinMonths' in df_fd_com.columns:
        df_fd_com['TerminMonths'] = df_fd_com['PeriodinMonths']

    df_fd_com.loc[df_fd_com['RateOfInterest'] < 1, 'RateOfInterest'] *= 100
    df_fd_com['DepositDate'] = pd.to_datetime(df_fd_com['DepositDate']).dt.strftime('%d-%m-%Y')
    df_fd_com['MaturityDate'] = pd.to_datetime(df_fd_com['MaturityDate']).dt.strftime('%d-%m-%Y')
    df_fd_com = df_fd_com[['ProductDescription','AdmissionNo','AccountNo','OperationTypeDesc','DepositDate','DepositAmount','TerminDays','TerminMonths','MaturityDate','MaturityAmount','RateOfInterest','InterestPaymentModeDesc','Status','PacsIDPkey','LedgerFolioNo','DepositTypeDesc','InterestAmount','IsInterestPosted','chkIsInterestPostingToCB','LastInterestPostingDate','TotalInterestAmount','JointAdmissionNo']]
    
    df_fd_com.to_excel(f'{uploadTemplatePath}/{templateName}_Upload_template.xlsx', index=False, sheet_name='FDnCumulative')    
    return print('Completed')

def generateKccTemplate(df, templateName):
    df_kcc = df    
    #loan_duplicates_kcc = df_kcc[df_kcc.duplicated('LoanNo', keep=False)]
    #loan_duplicates_kcc = df_kcc.groupby('ProductDescription')['LoanNo'].transform(lambda x: x.duplicated(keep=False))
    loan_duplicates_kcc = df_kcc[df_kcc.duplicated(['ProductDescription', 'LoanNo'], keep=False)]
    #loan_duplicates_kcc = loan_duplicates_kcc[loan_duplicates_kcc['LoanNo']]

    if not loan_duplicates_kcc.empty:
        loan_duplicates_kcc.to_excel(f'{basicValiidationFailPath}/{templateName}_Duplicate_Loan_Nos_template.xlsx', index=False, sheet_name='KccDuplicates')    
        
    df_kcc['SlNo'] = ''
    df_kcc['PacsId'] = pacIdKey
    df_kcc['BranchId'] = ''
    df_kcc.loc[(df_kcc['OutstandingInterest']).isna() | (df_kcc['OutstandingInterest'] == ''), 'OutstandingInterest'] = 0
    
    if 'LedgerFolioNo' not in df_kcc.columns:
        df_kcc['LedgerFolioNo'] = ''

    df_kcc.loc[df_kcc['ROIPercentage'] < 1, 'ROIPercentage'] *= 100
    df_kcc.loc[df_kcc['PenalROIPercentage'] < 1, 'PenalROIPercentage'] *= 100
    #df_kcc.loc[df_kcc['IODPercentage'] < 1, 'IODPercentage'] *= 100
    # Condition: Swap when 'A' is greater than 'B'
    condition = df_kcc['PenalROIPercentage'] > df_kcc['ROIPercentage']
    
    # Perform the swap based on the condition
    df_kcc.loc[condition, ['PenalROIPercentage', 'ROIPercentage']] = df_kcc.loc[condition, ['ROIPercentage', 'PenalROIPercentage']].values    
    df_kcc['IODPercentage'] = df_kcc['ROIPercentage'] + df_kcc['PenalROIPercentage']
    
    if 'CropDescription' not in df_kcc.columns and 'PurposeDescription' in df_kcc.columns:
        df_kcc['CropDescription'] = df_kcc['PurposeDescription']

    if 'LoanPeriod' not in df_kcc.columns and 'PeriodinMonths' in df_kcc.columns:
        df_kcc['LoanPeriod'] = df_kcc['PeriodinMonths']

    df_kcc['SanctionedDate'] = pd.to_datetime(df_kcc['SanctionedDate']).dt.strftime('%d-%m-%Y')
    df_kcc['DueDate'] = pd.to_datetime(df_kcc['DueDate']).dt.strftime('%d-%m-%Y')
    df_kcc.loc[(df_kcc['OutstandingInterest']).isna() | (df_kcc['OutstandingInterest'] == ''), 'OutstandingInterest'] = 0
    df_kcc_up = df_kcc[['SlNo', 'ProductDescription', 'AdmissionNo', 'LoanNo', 'Scheme', 'CropDescription', 'SanctionedDate','SanctionedAmount',
                                  'LoanPeriod','DueDate', 'ROIPercentage', 'PenalROIPercentage', 'IODPercentage', 'OutstandingPrincipal', 
                                  'OutstandingInterest', 'PacsId', 'BranchId']]
                                 
    # Save DataFrame to an Excel file
    df_kcc_up.to_excel(f'{uploadTemplatePath}/{templateName}_Upload_template.xlsx', index=False, sheet_name='KCC')
    selected_Disp_df_kcc = df_kcc    
    
    selected_Disp_df_kcc['OtherChargesDebitAmount'] = 0
    selected_Disp_df_kcc['CollectedPrincipal'] = 0
    selected_Disp_df_kcc['CollectedInterest'] = 0
    selected_Disp_df_kcc['CollectedPenalInterest'] = 0
    selected_Disp_df_kcc['CollectedIOD'] = 0
    selected_Disp_df_kcc['CollectedOthers'] = 0
    
    selected_Disp_df_kcc['BalancePenalInterest'] = 0
    selected_Disp_df_kcc['BalanceIOD'] = 0
    selected_Disp_df_kcc['Charges'] = 0    
    
    selected_Disp_df_kcc = selected_Disp_df_kcc[['ProductDescription', 'AdmissionNo', 'LoanNo', 'SanctionedDate','LedgerFolioNo','SanctionedAmount','OtherChargesDebitAmount',
                                                 'CollectedPrincipal','CollectedInterest','CollectedPenalInterest','CollectedIOD','CollectedOthers', 'OutstandingPrincipal', 
                                  'OutstandingInterest','BalancePenalInterest','BalanceIOD', 'Charges','PacsId', 'BranchId']].rename(columns={
                                  'SanctionedDate':'TransactionDate', 'SanctionedAmount':'DisbursementAmount',
                                  'OutstandingPrincipal':'BalancePrincipal', 'OutstandingInterest':'BalanceInterest'})
    
    selected_Disp_df_kcc.to_excel(f'{uploadTemplatePath}/{templateName}_Disbursal_Upload_template.xlsx', index=False, sheet_name='KCCDisbursal')
    return print('Completed')


def generateOtherLoansTemplate(df, templateName):
    df_other_loans = df     
    print(df_other_loans.columns)
    #loan_duplicates_other = df_other_loans[df_other_loans.duplicated('LoanNo', keep=False)]
    loan_duplicates_other = df_other_loans[df_other_loans.duplicated(['ProductDescription', 'LoanNo'], keep=False)]

    if not loan_duplicates_other.empty:
        loan_duplicates_other.to_excel(f'{basicValiidationFailPath}/{templateName}_Duplicate_Loan_Nos.xlsx', index=False, sheet_name='OherDuplicates')
    
    df_other_loans['SlNo'] = ''
    df_other_loans['PacsId'] = pacIdKey
    df_other_loans['BranchId'] = ''


    #df_other_loans[['ROIPercentage', 'PenalROIPercentage', 'IODPercentage']] = df_other_loans[['ROIPercentage', 'PenalROIPercentage', 'IODPercentage']].astype('float')
    
    df_other_loans.loc[df_other_loans['ROIPercentage'] < 1, 'ROIPercentage'] *= 100
    df_other_loans.loc[df_other_loans['PenalROIPercentage'] < 1, 'PenalROIPercentage'] *= 100
    #df_other_loans.loc[df_other_loans['IODPercentage'] < 1, 'IODPercentage'] *= 100
    # Perform the swap based on the condition
    condition = df_other_loans['PenalROIPercentage'] > df_other_loans['ROIPercentage']
    df_other_loans.loc[condition, ['PenalROIPercentage', 'ROIPercentage']] = df_other_loans.loc[condition, ['ROIPercentage', 'PenalROIPercentage']].values    
    df_other_loans['IODPercentage'] = df_other_loans['ROIPercentage'] + df_other_loans['PenalROIPercentage']
    df_other_loans = df_other_loans.rename(columns={'ROIPercentage':'ROI','PenalROIPercentage':'PenalROI','IODPercentage':'IOD'})
    
    df_other_loans.loc[(df_other_loans['OutstandingInterest']).isna() | (df_other_loans['OutstandingInterest'] == ''), 'OutstandingInterest'] = 0
    if 'LedgerFolioNo' not in df_other_loans.columns:
        df_other_loans['LedgerFolioNo'] = ''

    if 'LoanPeriod' not in df_other_loans.columns and 'PeriodinMonths' in df_other_loans.columns:
        df_other_loans['LoanPeriod'] = df_other_loans['PeriodinMonths']
    
    df_other_loans['SanctionedDate'] = pd.to_datetime(df_other_loans['SanctionedDate']).dt.strftime('%d-%m-%Y')
    df_other_loans['DueDate'] = pd.to_datetime(df_other_loans['DueDate']).dt.strftime('%d-%m-%Y')
    # Select specific columns and rename them using alias names
    df_other_loans = df_other_loans[['SlNo','ProductDescription','AdmissionNo','LoanNo','PurposeDescription','SanctionedDate','SanctionedAmount','LoanPeriod','DueDate','ROI','PenalROI','IOD','LedgerFolioNo','OutstandingPrincipal','OutstandingInterest','PacsId','BranchId']]
    
    df_other_loans.to_excel(f'{uploadTemplatePath}/{templateName}Loans_Upload_template.xlsx', index=False, sheet_name='OtherLoans')
    
    selected_Disp_df_other = df_other_loans
    
    selected_Disp_df_other['OtherChargesDebitAmount'] = 0
    selected_Disp_df_other['CollectedPrincipal'] = 0
    selected_Disp_df_other['CollectedInterest'] = 0
    selected_Disp_df_other['CollectedPenalInterest'] = 0
    selected_Disp_df_other['CollectedIOD'] = 0
    selected_Disp_df_other['CollectedOthers'] = 0    
    selected_Disp_df_other['BalancePenalInterest'] = 0
    selected_Disp_df_other['BalanceIOD'] = 0
    selected_Disp_df_other['Charges'] = 0
        
    selected_Disp_df_other = selected_Disp_df_other[['ProductDescription', 'AdmissionNo', 'LoanNo', 'SanctionedDate','LedgerFolioNo','SanctionedAmount','OtherChargesDebitAmount','CollectedPrincipal','CollectedInterest','CollectedPenalInterest','CollectedIOD','CollectedOthers', 'OutstandingPrincipal', 
                                  'OutstandingInterest','BalancePenalInterest','BalanceIOD', 'Charges','PacsId', 'BranchId']].rename(columns={
                                  'SanctionedDate':'TransactionDate', 'SanctionedAmount':'DisbursementAmount',
                                  'OutstandingPrincipal':'BalancePrincipal', 'OutstandingInterest':'BalanceInterest'})
    
    selected_Disp_df_other.to_excel(f'{uploadTemplatePath}/{templateName}Loan_Disbursal_Upload_template.xlsx', index=False, sheet_name='OtherLoanDisbursal')
    return print('Completed')


def generateLTMTTemplate(df, templateName):
    df_mt_lt_loans = df                
    #df_mt_lt_loans['IsDuplicate'] = df_mt_lt_loans.groupby('ProductDescription')['LoanNo'].transform(lambda x: x.duplicated(keep=False))
    loan_duplicates_mtlt = df_mt_lt_loans[df_mt_lt_loans.duplicated(['ProductDescription', 'LoanNo'], keep=False)]
    
    if not loan_duplicates_mtlt.empty:
        loan_duplicates_mtlt.to_excel(f'{basicValiidationFailPath}/{templateName}_Duplicate_Loan_Nos.xlsx', index=False, sheet_name='MtLtDuplicates')

    if 'LoanPeriod' not in df_mt_lt_loans.columns and 'PeriodinMonths' in df_mt_lt_loans.columns:
        df_mt_lt_loans['LoanPeriod'] = df_mt_lt_loans['PeriodinMonths']

    df_mt_lt_loans.loc[df_mt_lt_loans['ROIPercentage'] < 1, 'ROIPercentage'] *= 100
    df_mt_lt_loans.loc[df_mt_lt_loans['PenalROIPercentage'] < 1, 'PenalROIPercentage'] *= 100
    #df_mt_lt_loans.loc[df_mt_lt_loans['IODPercentage'] < 1, 'IODPercentage'] *= 100
    # Condition: Swap when 'A' is greater than 'B'
    condition = df_mt_lt_loans['PenalROIPercentage'] > df_mt_lt_loans['ROIPercentage']
    
    # Perform the swap based on the condition
    df_mt_lt_loans.loc[condition, ['PenalROIPercentage', 'ROIPercentage']] = df_mt_lt_loans.loc[condition, ['ROIPercentage', 'PenalROIPercentage']].values    
    df_mt_lt_loans['IODPercentage'] = df_mt_lt_loans['ROIPercentage'] + df_mt_lt_loans['PenalROIPercentage']
    
    df_mt_lt_loans['LCDate'] = ''
    df_mt_lt_loans['GestationPeriod'] = '0'
    df_mt_lt_loans['IODPercentage'] = df_mt_lt_loans['ROIPercentage'] + df_mt_lt_loans['PenalROIPercentage']
    df_mt_lt_loans['PacsId'] = pacIdKey
    df_mt_lt_loans['BranchId'] = ''
    df_mt_lt_loans['SlNo'] = ''
    
    if 'LedgerFolioNo' not in df_mt_lt_loans.columns:
        df_mt_lt_loans['LedgerFolioNo'] = ''

    df_mt_lt_loans['SanctionedDate'] = pd.to_datetime(df_mt_lt_loans['SanctionedDate']).dt.strftime('%d-%m-%Y')
    df_mt_lt_loans['DueDate'] = pd.to_datetime(df_mt_lt_loans['DueDate']).dt.strftime('%d-%m-%Y')
    df_mt_lt_loans = df_mt_lt_loans[['SlNo','AdmissionNo','ProductDescription','LoanNo','Scheme','PurposeDescription','SanctionedDate','SanctionedAmount',
                               'RepaymentFrequency','LoanPeriod','GestationPeriod','FirstInstallmentDate','InstallmentAmount',
                               'DueDate','ROIPercentage','PenalROIPercentage','IODPercentage','LANo','LCNo','LCDate','DCCBLoanAccountNo',
                               'OutstandingPrincipal','OutstandingInterest','PacsId','BranchId']]
    
    df_mt_lt_loans.to_excel(f'{uploadTemplatePath}/{templateName}Loans_Upload_template.xlsx', index=False, sheet_name='MTLoans')
    selected_Disp_df_mt_lt = df_mt_lt_loans
    
    selected_Disp_df_mt_lt['OtherChargesDebitAmount'] = 0
    selected_Disp_df_mt_lt['CollectedPrincipal'] = 0
    selected_Disp_df_mt_lt['CollectedInterest'] = 0
    selected_Disp_df_mt_lt['CollectedPenalInterest'] = 0
    selected_Disp_df_mt_lt['CollectedIOD'] = 0
    selected_Disp_df_mt_lt['CollectedOthers'] = 0    
    selected_Disp_df_mt_lt['BalancePenalInterest'] = 0
    selected_Disp_df_mt_lt['BalanceIOD'] = 0
    selected_Disp_df_mt_lt['Charges'] = 0    

    if 'LedgerFolioNo' not in selected_Disp_df_mt_lt.columns:
        selected_Disp_df_mt_lt['LedgerFolioNo'] = ''
        
    selected_Disp_df_mt_lt = selected_Disp_df_mt_lt[['AdmissionNo','ProductDescription', 'LoanNo', 'SanctionedDate','LedgerFolioNo','SanctionedAmount','OtherChargesDebitAmount','CollectedPrincipal','CollectedInterest','CollectedPenalInterest','CollectedIOD','CollectedOthers', 'OutstandingPrincipal', 
                                  'OutstandingInterest','BalancePenalInterest','BalanceIOD', 'Charges','PacsId', 'BranchId']].rename(columns={
                                  'SanctionedDate':'TransactionDate', 'SanctionedAmount':'DisbursementAmount',
                                  'OutstandingPrincipal':'BalancePrincipal', 'OutstandingInterest':'BalanceInterest'})
    
    selected_Disp_df_mt_lt.to_excel(f'{uploadTemplatePath}/{templateName}Loan_Disbursal_Upload_template.xlsx', index=False, sheet_name='MtLtLoanDisbursal')
    
    return print('Completed')

def generateTransactionTemplate(df,templateName):
    df_transactions = df
    
    df_transactions['OtherChargesDebitAmount'] = 0    
    df_transactions['DisbursementAmount'] = 0    
    df_transactions['BalancePenalInterest'] = 0
    df_transactions['BalanceIOD'] = 0
    df_transactions['Charges'] = 0    
    df_transactions['PacsId'] = pacIdKey
    df_transactions['BranchId'] = ''
    
    if 'LedgerFolioNo' not in df_transactions.columns:
        df_transactions['LedgerFolioNo'] = ''

    if 'CollectedIOD' not in df_transactions.columns:
        df_transactions['CollectedIOD'] = 0

    df_transactions['TransactionDate'] = pd.to_datetime(df_transactions['TransactionDate']).dt.strftime('%d-%m-%Y')    
    df_transactions = df_transactions[['AdmissionNo','ProductDescription', 'LoanNo', 'TransactionDate','LedgerFolioNo','DisbursementAmount','OtherChargesDebitAmount','CollectedPrincipal','CollectedInterest','CollectedPenalInterest','CollectedIOD','CollectedOthers', 'BalancePrincipal', 
                                  'BalanceInterest','BalancePenalInterest','BalanceIOD', 'Charges','PacsId', 'BranchId']]
    
    df_transactions.to_excel(f'{uploadTemplatePath}/{templateName}Loan_Transactions_Upload_template.xlsx', index=False, sheet_name='Transactions')
    print('Completed')


def checkAdmissionNosExists(dfm,dfo,colName,fileBegin):
    
    # Replace 'col1' and 'col2' with the names of your columns
    col1_values = dfo[colName]
    col2_values = dfm[colName]
    
    # Check which values in col1 exist in col2
    exists_in_col2 = col1_values.isin(col2_values)
    
    
    # Display values that do not exist in col2
    non_existing_values= col1_values[~exists_in_col2]
    non_existing_admission_df = pd.DataFrame(non_existing_values.tolist())
    #non_existing_values_df - pd.DataFrame(non_existing_valuesss, columns=['ID', 'Name'])
    
    # Check if the DataFrame is not empty
    if not non_existing_admission_df.empty:
        # Save the DataFrame to an Excel file
        file_path = f'{basicValiidationFailPath}/{fileBegin}_Admission_No_Non_Existing.xlsx'
        non_existing_admission_df.to_excel(file_path, index=False)
        return print(f"The Admission Nos not exist saved to {file_path}")
    else:
        return print(f"{fileBegin} Admission Nos all are exist.")


def checkAdmSanDates(df,df_other,dfCol1,dfCol2,df_otherCol2,fileBegin):
    # Inner join on the common column 'id'
    merge_df = pd.merge(df[[dfCol1, dfCol2]], df_other[[dfCol1, df_otherCol2]], on=dfCol1, how='inner')

    # Apply filter conditions similar to SQL's WHERE clause
    filtered_adm_san_date_df = merge_df[(merge_df[dfCol2] > merge_df[df_otherCol2])]

    # Check if the DataFrame is not empty
    if not filtered_adm_san_date_df.empty:
        # Save the DataFrame to an Excel file
        file_path = f'{basicValiidationFailPath}/{fileBegin}_SanctionOrDepositDate_lessthan_AdmissionDate.xlsx'
        filtered_adm_san_date_df.to_excel(file_path, index=False)
        # Find the index of the minimum date for each value
        idx = filtered_adm_san_date_df.groupby(dfCol1)[df_otherCol2].idxmin()
        
        # Use these indices to filter the DataFrame
        df_min = filtered_adm_san_date_df.loc[idx]
        
        # Reset the index if needed
        df_min = df_min.reset_index(drop=True)
        # Create a mapping Series
        mapping = df_min.set_index(dfCol1)[df_otherCol2]
        
        # Update 'value1' column based on 'id' mapping
        df[dfCol2] = df[dfCol1].map(mapping).combine_first(df[dfCol2])
        return print(f"Sanction Or Deposit Dates saved to {file_path}")
    else:
        return print(f"{fileBegin} Sanction Or Deposit Dates are fine.")
        
# Define a custom function with more complex conditions
def rename_meber_columns(col_name):
    if 'customer' in col_name:
        return 'CustomerTypeDescription'
    elif col_name == 'member type' or col_name == 'membertypedescription':
        return 'MemberTypeDescription'
    elif col_name == 'admission no' or 'admissionno' in col_name or col_name == 'admno':
        return 'AdmissionNo'
    elif col_name == 'surname' or col_name == 'membersurname':
        return 'MemberSurName'
    elif col_name == 'member name' or col_name == 'membername':
        return 'MemberName'
    elif 'gender' in col_name:
        return 'GenderDescription'
    elif 'share' in col_name:
        return 'ShareBalance'
    elif 'village' in col_name:
        return 'VillageDescription'
    elif 'ledger' in col_name:
        return 'LedgerFolioNo'
    elif col_name == 'admission date' or col_name == 'admissiondate':
        return 'AdmissionDate'
    elif 'age' in col_name:
        return 'Age'
    elif 'birth' in col_name or 'dob' in col_name:
        return 'DOB'
    elif col_name == 'father name' or col_name == 'father' or col_name == 'fathername' or col_name == 'father / mother name':
        return 'FatherName'
    elif 'marital' in col_name:
        return 'MaritalStatusDesc'
    elif col_name == 'spouse name' or col_name == 'spouse' or col_name == 'spousename':
        return 'SpouseName'
    elif 'mobile' in col_name or 'contact' in col_name:
        return 'ContactNo'
    elif 'aadhar' in col_name or 'adhar' in col_name or 'adhaar' in col_name:
        return 'AdhaarCardNo'
    elif 'caste' in col_name:
        return 'CasteDescription'
    elif 'community' in col_name:
        return 'CommunityDescription'
    elif 'farmer' in col_name:
        return 'FarmerTypeDescription'
    elif 'dccb' in col_name:
        return 'DCCBSBACNO'
    return col_name

def rename_other_columns(col_name):
    if 'product' in col_name or col_name == 'prdesc':
        return 'ProductDescription'
    elif col_name == 'admission no' or col_name == 'admissionno' or col_name == 'agent admission no' or col_name == 'admno':
        return 'AdmissionNo'
    elif col_name == 'account no' or col_name == 'accountno':
        return 'AccountNo'
    elif 'deposit date' in col_name or 'depositdate' in col_name:
        return 'DepositDate'
    elif 'sanction date' in col_name or 'sanctioneddate' in col_name or col_name == 'sancdate':
        return 'SanctionedDate'
    elif col_name == 'balance':
        return 'Balance'
    elif 'ledger' in col_name or col_name == 'ledg':
        return 'LedgerFolioNo'
    elif 'cheque' in col_name:
        return 'ChequeOption'
    elif 'operation' in col_name:
        return 'OperationTypeDesc' 
    elif 'organization' in col_name or 'organisation' in col_name:
        return 'IsOrganisation' 
    elif 'register' in col_name:
        return 'RegisterSlNo' 
    elif 'years' in col_name:
        return 'PeriodinYears' 
    elif 'months' in col_name:
        return 'PeriodinMonths' 
    elif 'rate of interest' in col_name:
        return 'RateOfInterest' 
    elif 'total amount collected' in col_name:
        return 'TotalAmount'
    elif 'installment amount' in col_name:
        return 'InstallmentAmount' 
    elif col_name == 'maturity date':
        return 'MaturityDate'
    elif 'frequency' in col_name:
        return 'FrequencyDescription'
    elif 'transaction date' in col_name or 'transactiondate' in col_name:
        return 'TransactionDate'
    elif 'loanno' in col_name or 'loan no' in col_name or col_name == 'acno':
        return 'LoanNo'       
    elif col_name == 'scheme' or col_name == 'scheme1':
        return 'Scheme'
    elif 'crop / purpose' in col_name or col_name == 'purposedescription' or col_name == 'prpdesc':
        return 'PurposeDescription'    
    elif col_name == 'sanction amount' or col_name == 'sanctionedamount' or col_name == 'sancamt':
        return 'SanctionedAmount'
    elif col_name == 'due date' or col_name == 'duedate':
        return 'DueDate'
    elif col_name == 'due date':
        return 'DueDate'
    elif col_name == 'roi' or col_name == 'intrate':
        return 'ROIPercentage'
    elif col_name == 'penal roi' or col_name == 'penalroi' or col_name == 'penrate':
        return 'PenalROIPercentage'
    elif col_name == 'roi after due date' or col_name == 'iod' or col_name == 'iodrate':
        return 'IODPercentage'
    elif col_name == 'repayment mode' or col_name == 'rpfreq':
        return 'RepaymentFrequency'
    elif 'gestation' in col_name or col_name == 'gstperiod':
        return 'GestationPeriod' 
    elif col_name == 'first installment due date' or col_name == 'fstinstdt':
        return 'FirstInstallmentDate'
    elif col_name == 'installment amount' or col_name == 'instamt':
        return 'InstallmentAmount'
    elif col_name == 'la no' or col_name == 'lano':
        return 'LANo'
    elif col_name == 'lc no' or col_name == 'lcno':
        return 'LCNo'
    elif col_name == 'lc date' or col_name == 'lcdate':
        return 'LCDate'
    elif 'dccb' in col_name:
        return 'DCCBLoanAccountNo'
    elif col_name == 'outstanding interest (rs.)' or col_name == 'outstandinginterest' or col_name == 'osint':
        return 'OutstandingInterest'
    elif col_name == 'outstanding principal (rs.)' or col_name == 'outstandingprincipal' or col_name == 'osbal':
        return 'OutstandingPrincipal'
    elif 'loanperiod' in col_name or 'loan period' in col_name or col_name == 'lnprd':
        return 'LoanPeriod'
    elif col_name == 'collectedprincipal':
        return 'CollectedPrincipal'
    elif col_name == 'collectedinterest':
        return 'CollectedInterest'
    elif col_name == 'collectedpenalinterest':
        return 'CollectedPenalInterest'
    elif col_name == 'collectediod':
        return 'CollectedIOD'
    elif col_name == 'collectedothers':
        return 'CollectedOthers'
    elif col_name == 'balanceprincipal':
        return 'BalancePrincipal'
    elif col_name == 'balanceinterest':
        return 'BalanceInterest'
    elif col_name == 'interest amount':
        return 'InterestAmount'
    elif col_name == 'deposit amount':
        return 'DepositAmount'
    elif 'days' in col_name:
        return 'TerminDays'
    elif col_name == 'maturity amount':
        return 'MaturityAmount'
    elif col_name == 'deposit type':
        return 'DepositTypeDesc'
    elif col_name == 'interest/installment mode':
        return 'InterestPaymentModeDesc'
    elif col_name == 'in interest add to deposit?':
        return 'IsInterestPosted'
    elif col_name == 'last interest posting date':
        return 'LastInterestPostingDate'
    elif col_name == 'total interest amount':
        return 'TotalInterestAmount'    
    return col_name
        

# Print the names of the sheets and the first few rows of each dataframe
for sheet_name, df in sheetsDict.items():
    #print(f"Sheet Names df_{sheet_name.lower().replace(' ','_').replace('.','_')}")
    # Create a dynamic variable name by concatenating a string
    #var_name = f"df_{sheet_name.lower().replace(' ','_').replace('.','_')}"
    #globals()[var_name] = df
    df = df.dropna(how='all')    
    df.columns = df.columns.str.lower().str.rstrip('.')
    if sheet_name == 'Membership':
        # Apply the custom function to rename columns
        df.columns = [rename_meber_columns(col) for col in df.columns]
        df = df[df['AdmissionNo'] != 0]
        # Identify and display duplicate rows based on specific columns
        duplicates = df[df.duplicated(['AdmissionNo'])]
        
        # Check if the DataFrame is not empty
        if not duplicates.empty:
            # Save the DataFrame to an Excel file
            file_path = f'{folderPath}/BasicValidationFail/Membership_Details_Duplicates_With_Admission.xlsx'
            duplicates.to_excel(file_path, index=False)
            print(f"The DataFrame was saved to {file_path}")
        else:
            print("The DataFrame is empty and was not saved.")

        cutOffDate = datetime(2024, 3, 31)        
        # Get rows where column 'B' is greater than the threshold
        df['AdmissionDate'] = pd.to_datetime(df['AdmissionDate'])
        cutOffDate_df = df[df['AdmissionDate'] > cutOffDate]
        
        # Check if the DataFrame is not empty
        if not cutOffDate_df.empty:
            # Save the DataFrame to an Excel file
            file_path = f'{folderPath}/BasicValidationFail/Membership_Details_cut_off_date_Admission.xlsx'
            cutOffDate_df.to_excel(file_path, index=False)
            print(f"The DataFrame was saved to {file_path}")
        else:
            print("The DataFrame is empty and was not saved.")

        admission_date_nan_df = df[df['AdmissionDate'].isna()]
        # Check if the DataFrame is not empty
        if not admission_date_nan_df.empty:
            # Save the DataFrame to an Excel file
            file_path = f'{folderPath}/BasicValidationFail/Membership_Details_nan_Admission_date.xlsx'
            admission_date_nan_df.to_excel(file_path, index=False)
            print(f"The DataFrame was saved to {file_path}")
        else:
            print("The DataFrame is empty and was not saved.")
        admission_date_empty_df = df[df['AdmissionDate'] == '']
        # Check if the DataFrame is not empty
        if not admission_date_empty_df.empty:
            # Save the DataFrame to an Excel file
            file_path = f'{folderPath}/BasicValidationFail/Membership_Details_empty_Admission_date.xlsx'
            admission_date_empty_df.to_excel(file_path, index=False)
            print(f"The DataFrame was saved to {file_path}")
        else:
            print("The DataFrame is empty and was not saved.")

        membership = df
        print(membership.columns)
    else:
        df.columns = [rename_other_columns(col) for col in df.columns]
        df = df[df['AdmissionNo'] != 0]
        checkAdmissionNosExists(membership,df,column_name,sheet_name)
        if sheet_name in ['SB','SHG','Pigmy','FD-C','FD-NC','rd']:
            checkAdmSanDates(membership,df,column_name, admissionDate, depositDate,sheet_name)
            if sheet_name == 'SB':
                generateSbTemplate(df,'SBDeposit')
            elif sheet_name == 'SHG':
                generateSbTemplate(df,'SHGDeposit')
            elif sheet_name == 'Pigmy':
                generatePigmyTemplate(df,'PigmyDeposit')
            elif sheet_name == 'FD-NC':
                generateFdNonTemplate(df,'FDNonCummulative')
            elif sheet_name == 'rd':
                generateFdNonTemplate(df,'RD')    
            elif sheet_name == 'FD-C':
                generateFdComTemplate(df,'FDCummulative')    
        elif 'Transaction' in sheet_name or 'Disbursal' in sheet_name:
            checkAdmSanDates(membership,df,column_name, admissionDate, transactionDate,sheet_name)
            if 'Transaction' in sheet_name:
                generateTransactionTemplate(df,sheet_name)
        elif 'LTMT' in sheet_name or 'other' in sheet_name or 'kcc' in sheet_name:            
            checkAdmSanDates(membership,df,column_name, admissionDate, sanctionDate,sheet_name)
            if sheet_name == 'kcc':
                generateKccTemplate(df,sheet_name)
            elif 'other' in sheet_name:
                generateOtherLoansTemplate(df,sheet_name)
            elif 'LTMT' in sheet_name:
                generateLTMTTemplate(df,sheet_name)    

        
print('All Complete')
#print(sheetsDict['Membership'].columns)

The DataFrame is empty and was not saved.
The DataFrame is empty and was not saved.
The DataFrame is empty and was not saved.
The DataFrame is empty and was not saved.
Index(['CustomerTypeDescription', 'MemberTypeDescription', 'AdmissionNo',
       'MemberSurName', 'MemberName', 'GenderDescription', 'ShareBalance',
       'VillageDescription', 'LedgerFolioNo', 'AdmissionDate', 'Age', 'DOB',
       'FatherName', 'MaritalStatusDesc', 'SpouseName', 'address', 'ContactNo',
       'AdhaarCardNo', 'CasteDescription', 'CommunityDescription',
       'FarmerTypeDescription', 'dividend amount', 'thrift balance',
       'DCCBSBACNO'],
      dtype='object')
The Admission Nos not exist saved to E:/Evaidya/PAC/Nerali/BasicValidationFail/Land Details_Admission_No_Non_Existing.xlsx
Personal Loan other Admission Nos all are exist.
Personal Loan other Sanction Or Deposit Dates are fine.
Index(['AdmissionNo', 'loan type', 'ProductDescription', 'LoanNo',
       'PurposeDescription', 'LedgerFolioNo', 'Sche

C:\Users\ILYAS\AppData\Local\Temp\ipykernel_12936\2248067783.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_Disp_df_mt_lt['OtherChargesDebitAmount'] = 0
C:\Users\ILYAS\AppData\Local\Temp\ipykernel_12936\2248067783.py:285: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_Disp_df_mt_lt['CollectedPrincipal'] = 0
C:\Users\ILYAS\AppData\Local\Temp\ipykernel_12936\2248067783.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

Completed
kcc Admission Nos all are exist.
kcc Sanction Or Deposit Dates are fine.
Completed
SB Admission Nos all are exist.
SB Sanction Or Deposit Dates are fine.
Complete
FD-C Admission Nos all are exist.
FD-C Sanction Or Deposit Dates are fine.
Completed
Collateral Details Admission Nos all are exist.
Transaction Details Admission Nos all are exist.
Transaction Details Sanction Or Deposit Dates are fine.
Completed
Pigmy Admission Nos all are exist.
Pigmy Sanction Or Deposit Dates are fine.
Completed
The Admission Nos not exist saved to E:/Evaidya/PAC/Nerali/BasicValidationFail/Agent Details_Admission_No_Non_Existing.xlsx
All Complete


In [14]:
from datetime import date

def age(born): 
    #born = datetime.strptime(born, "%Y-%m-%d").date() 
    today = date.today() 
    return today.year - born.year - ((today.month,  
                                      today.day) < (born.month,  
                                                    born.day))
# Membership Updates date of birth empty update
df_membership = membership
# Function to add 20 years to a date
def sub_20_years(dob):
    return dob - pd.DateOffset(years=20)

# Update 'admission_date' where 'date_of_birth' is empty
df_membership.loc[df_membership['DOB'].isna() | (df_membership['DOB'] == ''), 'DOB'] = df_membership['AdmissionDate'].apply(sub_20_years)

df_membership['calculated_age'] = df_membership['DOB'].apply(age)

    
# Update 'admission_date' where 'date_of_birth' is empty
df_membership.loc[df_membership['calculated_age'].isna() | (df_membership['calculated_age'] < 18), 'DOB'] = df_membership['AdmissionDate'].apply(sub_20_years)
df_membership.loc[df_membership['Age'].isna() | (df_membership['Age'] == ''), 'Age'] = df_membership['DOB'].apply(age)

df_membership.loc[df_membership['MemberTypeDescription'].str.contains('A', case=False, na=False), 'MemberTypeDescription'] = 'A Type'
df_membership.loc[df_membership['MemberTypeDescription'].str.contains('B', case=False, na=False), 'MemberTypeDescription'] = 'B Type'
df_membership.loc[df_membership['MemberTypeDescription'].str.contains('C', case=False, na=False), 'MemberTypeDescription'] = 'C Type'


df_membership.loc[df_membership['MemberTypeDescription'] == 'A Type', 'CustomerTypeDescription'] = 'Member'
df_membership.loc[df_membership['MemberTypeDescription'] == 'B Type', 'CustomerTypeDescription'] = 'Nominal Member'
df_membership.loc[df_membership['MemberTypeDescription'] == 'C Type', 'CustomerTypeDescription'] = 'Organisation'

membership['SpouseName'] = membership['SpouseName'].astype(str)
# Regex pattern to match special characters
pattern = r'[^\w\s]'

#special_chars_of_surname = membership['MemberName'].str.contains(pattern).sum()
#print(special_chars_of_surname)

# Replace special characters with space and trim the resulting strings
#membership['MemberName'] = membership['MemberName'].str.replace(pattern, ' ', regex=True).str.strip()
#membership['FatherName'] = membership['FatherName'].str.replace(pattern, ' ', regex=True).str.strip()
#membership['SpouseName'] = membership['SpouseName'].str.replace(pattern, ' ', regex=True).str.strip()
#membership['MemberSurName'] = membership['MemberSurName'].str.replace(pattern, ' ', regex=True).str.strip()

membership['MemberName'] = membership['MemberName'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
membership['FatherName'] = membership['FatherName'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
membership['SpouseName'] = membership['SpouseName'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
membership['MemberSurName'] = membership['MemberSurName'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

df_membership.loc[(df_membership['SpouseName'].isna() | (df_membership['SpouseName'] == 'nan')), 'SpouseName'] = ''

df_membership.loc[df_membership['SpouseName'].isna() | (df_membership['SpouseName'] == '') &  (df_membership['MaritalStatusDesc'] == 'Married')  & (df_membership['FatherName']).notna() & (df_membership['FatherName'] != ''), 'SpouseName'] = df_membership['FatherName']
df_membership.loc[df_membership['SpouseName'].isna() | (df_membership['SpouseName'] == ''),'SpouseName'] = df_membership['FatherName']

df_membership.loc[df_membership['SpouseName'].isna() | (df_membership['SpouseName'] == ''),'SpouseName'] = 'NA'
df_membership.loc[df_membership['FatherName'].isna() | (df_membership['FatherName'] == ''),'FatherName'] = 'NA'
df_membership['ContactNo'] = df_membership['ContactNo'].astype(str)
df_membership['AdhaarCardNo'] = df_membership['AdhaarCardNo'].astype(str)

df_membership['ContactNo'] = df_membership['ContactNo'].str.rstrip('.0')
df_membership['AdhaarCardNo'] = df_membership['AdhaarCardNo'].str.rstrip('.0')
df_membership.loc[(df_membership['ContactNo'].isna()) | (df_membership['ContactNo'].str.len() != 10), 'ContactNo'] = ''
df_membership.loc[(df_membership['AdhaarCardNo'].isna()) | (df_membership['AdhaarCardNo'].str.len() != 12), 'AdhaarCardNo'] = ''

df_membership.loc[(df_membership['DCCBSBACNO']).isna() | (df_membership['DCCBSBACNO'] == ''), 'DCCBSBACNO'] = 0

df_membership.loc[(df_membership['MemberSurName'] == '') | (df_membership['MemberSurName']).isna() & (df_membership['GenderDescription'] == 'Others'), 'MemberSurName'] = df_membership['MemberName'].str[0]
df_membership.loc[(df_membership['MemberSurName'] == '') | (df_membership['MemberSurName']).isna() & (df_membership['GenderDescription'] == 'Male'), 'MemberSurName'] = 'Mr'
df_membership.loc[(df_membership['MemberSurName'] == '') | (df_membership['MemberSurName']).isna() & (df_membership['GenderDescription'] == 'Female'), 'MemberSurName'] = 'Mrs'

df_membership['MemberNameRegional'] = ''
df_membership['FatherNameinRegional'] = ''
df_membership['SpouseNameinRegional'] = ''
df_membership['Address2'] = ''
df_membership['ThriftBalance'] = ''
df_membership['DividentBalance'] = ''

df_membership['PacsIDPKey'] = pacIdKey
df_membership['BranchId'] = ''
df_membership['Address1'] = df_membership['VillageDescription']

df_membership['Age'] = df_membership['calculated_age']

df_membership['AdmissionDate'] = pd.to_datetime(df_membership['AdmissionDate']).dt.strftime('%d-%m-%Y')
df_membership['DOB'] = pd.to_datetime(df_membership['DOB']).dt.strftime('%d-%m-%Y')
df_membership = df_membership[['CustomerTypeDescription','MemberTypeDescription','AdmissionNo','MemberSurName','MemberName','FatherName',
                               'SpouseName','MemberNameRegional','FatherNameinRegional','SpouseNameinRegional','DOB','Age','AdmissionDate','GenderDescription','MaritalStatusDesc','CommunityDescription','CasteDescription','FarmerTypeDescription','Address1','Address2','VillageDescription','LedgerFolioNo','ContactNo','ShareBalance','ThriftBalance','DividentBalance','AdhaarCardNo','DCCBSBACNO','PacsIDPKey','BranchId']]

# Save DataFrame to an Excel file
df_membership.to_excel(f'{uploadTemplatePath}/MemberShip_Upload_template.xlsx', index=False, sheet_name='Membership_Details')

print('Completed')

Completed
